In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
with open("../out/gutenberg_chunked/test01_embeddings_20240109_153150.pickle", "rb") as f:
    data_raw = pickle.load(f)

datasize = 10000

# select 10000 datapoints randomly 

#np.random.seed(42)
#random_index = np.random.choice(len(data_raw), datasize, replace=False)

#data_raw = [data_raw[i] for i in tqdm(random_index)]

data_raw = data_raw[:datasize]


In [3]:
embeddings_lis = [elem["embedding"] for elem in data_raw]
authors = [elem["metadata"]["author"] for elem in data_raw]

del data_raw

In [4]:
df = pd.DataFrame(embeddings_lis)
df["author"] = authors

del embeddings_lis
del authors

In [5]:
train_data = df.sample(frac=0.8, random_state=0)
test_data = df.drop(train_data.index)

In [6]:
from autogluon.tabular import TabularDataset, TabularPredictor

c:\Users\arik_\Documents\Dokumente\Studium\Master\DataSecurity\EmbeddingAttacks\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
train_data["author"].groupby(train_data["author"]).count().sort_values(ascending=False)

author
Shakespeare, William             4827
Dante Alighieri                  2468
Wells, H. G. (Herbert George)     370
Parkman, Francis                  189
Bower, B. M.                      146
Name: author, dtype: int64

In [10]:
test_data["author"].groupby(test_data["author"]).count().sort_values(ascending=False)

author
Shakespeare, William             1214
Dante Alighieri                   609
Wells, H. G. (Herbert George)      84
Parkman, Francis                   54
Bower, B. M.                       39
Name: author, dtype: int64

In [11]:
predictor = TabularPredictor(label='author', problem_type='multiclass')
predictor.fit(train_data, time_limit=60*5, presets = 'medium_quality')


No path specified. Models will be saved in: "AutogluonModels\ag-20240109_162433"
Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "AutogluonModels\ag-20240109_162433"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
CPU Count:          8
Memory Avail:       7.71 GB / 15.71 GB (49.1%)
Disk Space Avail:   32.97 GB / 474.72 GB (6.9%)
Train Data Rows:    8000
Train Data Columns: 1536
Label Column:       author
Problem Type:       multiclass
Preprocessing data ...
Train Data Class Count: 5
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    7880.92 MB
	Train Data (Original)  Memory Usage: 93.75 MB (1.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata

In [ ]:
predictor.evaluate(test_data, silent=False)

Evaluation: accuracy on test data: 0.978
Evaluations on test data:
{
    "accuracy": 0.978,
    "balanced_accuracy": 0.9674345409220146,
    "mcc": 0.9590837491593869
}


{'accuracy': 0.978,
 'balanced_accuracy': 0.9674345409220146,
 'mcc': 0.9590837491593869}